In [1]:
import pandas,numpy
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import *
from IPython.display import display
import pyecharts
from pyecharts import options as opts
from pyecharts.charts import Bar,Map,Line3D,Line,Bar3D
from pyecharts.globals import ThemeType

In [2]:
engine = create_engine('postgresql://{0}:{1}@{2}:5432/{3}'.format("postgres",
                                                                    "postgres",
                                                                  "localhost",
                                                                  "postgres"))

In [4]:
df = pandas.read_sql_query("SELECT name FROM public.citieslocation",engine)

In [5]:
def showecharts(data) -> Bar:
    bar = pyecharts.charts.Bar(init_opts=opts.InitOpts(theme=ThemeType.ESSOS))
    bar.add_xaxis(data["date"].tolist())
    x = True
    for i in data:
        if i == "hour" or i == "name" or i == "date":pass
        else:
            bar.add_yaxis(i, data[i].tolist(),is_selected=x)
        x= False
    bar.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    bar.set_global_opts(title_opts=opts.TitleOpts(title=c.value),
                       xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=90)))
    return bar

def showecharts3d(data) -> Bar3D:
    x = ["{0}".format(i) for i in data["date"].tolist()]
    y = []
    for i in data:
        if i == "hour" or i == "name" or i == "date":pass
        else:
            y.append(i)
    data = [(x[i], y[j], data[y[j]].iloc[i]) for i in range(len(x)) for j in range(len(y))]
    bar = pyecharts.charts.Bar3D(init_opts=opts.InitOpts(theme=ThemeType.ESSOS))
    bar.add("{0} {1} {2}".format(c.value,m.value,h.value),
            [[d[0], d[1], d[2]] for d in data])
    return bar

def showechartsLine(data) -> Line:
    bar = pyecharts.charts.Line(init_opts=opts.InitOpts(theme=ThemeType.ESSOS))
    bar.add_xaxis(["{0}".format(i) for i in data["date"].tolist()])
    for i in data.columns:
        if i == "hour" or i == "name" or i == "date":pass
        else:
            bar.add_yaxis(i, data[i].tolist())

    bar.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    bar.set_global_opts(title_opts=opts.TitleOpts(title=c.value),
                       xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=90)))

    return bar

In [6]:
def queryData(m,h,c):
    m1 = m * 100
    m2 = (m + 1) * 100
    sql = "SELECT * \
    FROM public.chinacitiesair where name = '{0}' \
    and date > {1} and date < {2} and hour = {3}".format(c,m1,m2,h)
    return pandas.read_sql_query(sql,engine)

In [7]:
import IPython.display

In [8]:
def btnClick(i):
    IPython.display.clear_output()
    display(form)
    name = c.value
    if name == "输入":
        name = c1.value
    else:
        c1.value = name
    res = queryData(m.value,h.value,name)
    if tp.value == "2D柱状图":
        charts = showecharts(res)
    elif tp.value == "3D柱状图":
        charts = showecharts3d(res)
    else:
        charts = showechartsLine(res)
    display(charts.render_notebook())

In [9]:
tp = widgets.RadioButtons(
    options=['2D柱状图','3D柱状图','折线图'],
    disabled=False
)
b = Button(description="查询")
m = IntSlider(min=201801, max=201811)
h = IntSlider(min=0, max=23)
c = Dropdown(options=["输入"]+df["name"].tolist())
c1 = Text()
b.on_click(btnClick)
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)
form_items = [
    Box([Label(value='专题图样式'),tp] ,layout=form_item_layout),
    Box([Label(value='月份'),m] ,layout=form_item_layout),
    Box([Label(value='时间'), h], layout=form_item_layout),
    Box([Label(value='城市'),c,c1], layout=form_item_layout),
    b
]
form = Box(form_items, layout=Layout(
    display='flex',flex_flow='column',order='solid 2px',
    align_items='stretch',width='50%'))
form

Box(children=(Box(children=(Label(value='专题图样式'), RadioButtons(options=('2D柱状图', '3D柱状图', '折线图'), value='2D柱状图…